# Sentiment Analysis using the New York Times API

In [1]:
# Import statements
import csv
import pandas as pd
import time
%matplotlib inline

In [2]:
# Using API tool with key
from nytimesarticle import articleAPI
api = articleAPI('51ae5c44eb962681341060ede81808b8:11:73610715')

In [3]:
# Read in the dates to use for sentiment analysis
dframe=pd.read_csv('data/IYZ.csv')
date_list = list(dframe['date'])
cleaned_dates = []
for entry in date_list:
    cleaned_dates.append(entry.replace('-', ''))

In [4]:
# This function takes in a response to the NYT api and parses the articles into a list of dictionaries
def parse_articles(articles, datestamp):  
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['date'] = datestamp
        dic['text'] = i['headline']['main'].encode("utf8")
        if i['snippet'] is not None:
            dic['text'] = dic['text'] + " " + i['snippet'].encode("utf8")     
        news.append(dic)
    return(news) 

In [5]:
# This function accepts a list of dates and returns a dictionary of parsed articles for those dates 
def get_articles(dates,query):
    all_articles = []
    for date in dates:
        articles = api.search(q = query,
                fq = {'news_desk':['Business','Financial', 'Jobs','Retail','Outlook','Personal Investing','Technology','Wealth']},
                begin_date = date,
                end_date = date,
                sort='oldest')
        articles = parse_articles(articles,date)
        if len(articles) != 0:
            all_articles  = all_articles + articles
        time.sleep(0.1)
    return(all_articles)

In [6]:
# Verizon_articles
verizon_articles = get_articles(cleaned_dates,'Verizon')
verizon_df = pd.DataFrame.from_dict(verizon_articles)

In [7]:
# Store dataframe in a CSV for future analysis
verizon_df.to_csv('data/verizon.csv')

In [12]:
# Read the CSV after manual classification of text as positive or negative
verizon_df = pd.read_csv('data/verizon.csv')

In [13]:
verizon_df

,date,text
0,20100104,Firms Selling Apps for Simple Phones Software ...
1,20100104,"Watching TV Together, Miles Apart Technology n..."
2,20100104,"The Smartphone Makes and Breaks In a decade, c..."
3,20100105,Google Moves to Keep Its Lead as Web Goes Mobi...
4,20100106,"Google Introduces Nexus One, Its Rival to the ..."
5,20100106,Google Shakes but Doesn��t Upend the Cellphone...
6,20100111,Optimism Takes Charge at Consumer Electronics ...
7,20100114,The Fine Print Behind Google's Nexus One Googl...
8,20100115,Burst of Mobile Giving Adds Millions in Relief...
9,20100119,"Big, in Banks, Is in the Eye of the Beholder T..."
